In [4]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', 105)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
sns.set()

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore")

from subprocess import check_output

In [5]:
#モデル分割で使われる交差検証の数
nr_cv = 5

#SalePriceと特徴のlog値を使うためのスイッチ
use_logvals = 1

#相関で用いられるtarget
target = "SalePrice_Log"

#この閾値はpart3におけるML Regressorsで用いられる
min_val_corr = 0.4

#すでに使われるその他に似ていて、高い相関を見せる列を削除するスイッチ
drop_similar = 1

Some Useful Function

In [6]:
def get_best_score(grid):
    best_score = np.sqrt(-grid.best_score_)
    print(best_score)
    print(grid.best_params_)
    print(grid.best_estimator_)
    
    return best_score

def print_cols_large_corr(df, nr_c, targ):
    corr = df.corr()
    corr_abs = corr.abs()
    print (corr_abs.nlargest(nr_c, targ)[targ])
    
def plot_corr_matrix(df, nr_c, targ):
    corr = df.corr()
    corr_abs = corr.abs()
    cols = corr_abs.nlargest(nr_c, targ)[targ].index
    cm = np.corrcoef(df[cols].values.T)
    
    plt.figure(figsize=(nr_c/1.5, nr_c/1.5))
    sns.set(font_scale=1.25)
    sns.heatmap(cm, linewidth=1.5, annot=True, square=True,
               fmt=".2f", annot_kws={"size": 10},
               yticklabels=cols.values, xticklabels=cols.values
               )
    plt.show()

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")